In [2]:
# For making plots for the paper
import matplotlib.pyplot as plt
plt.style.use("seaborn-poster")
plt.rcParams["figure.figsize"] = (10, 6) # 5/3

In [18]:
N = 128
NP = 7 
NCORES = 16
P = NP * NCORES


lower_bounds = []
upper_bounds = []
ub_diff_lb_greater_than_one = []

# In some cases the result of the mulplication is a bit strange...
# example, if N//P is always 1, how is it that a range like (30, 32) is possible?
for rank in range(P):
    lb = (rank)*(N//P)  # This gives the intended loads where the last load is unbalanced
    ub = (rank+1)*(N//P) if rank != P-1 else N
    lower_bounds.append(lb)
    upper_bounds.append(ub)

    if ub - lb > 1:
        print(rank, lb, ub)
        ub_diff_lb_greater_than_one.append((lb, ub))

ub_diff_lb_greater_than_one

111 111 128


[(111, 128)]

In [16]:
for p in zip(lower_bounds, upper_bounds):
    print(p)

(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 5)
(5, 6)
(6, 8)
(8, 9)
(9, 10)
(10, 11)
(11, 12)
(12, 13)
(13, 14)
(14, 16)
(16, 17)
(17, 18)
(18, 19)
(19, 20)
(20, 21)
(21, 22)
(22, 24)
(24, 25)
(25, 26)
(26, 27)
(27, 28)
(28, 29)
(29, 30)
(30, 32)
(32, 33)
(33, 34)
(34, 35)
(35, 36)
(36, 37)
(37, 38)
(38, 40)
(40, 41)
(41, 42)
(42, 43)
(43, 44)
(44, 45)
(45, 46)
(46, 48)
(48, 49)
(49, 50)
(50, 51)
(51, 52)
(52, 53)
(53, 54)
(54, 56)
(56, 57)
(57, 58)
(58, 59)
(59, 60)
(60, 61)
(61, 62)
(62, 64)
(64, 65)
(65, 66)
(66, 67)
(67, 68)
(68, 69)
(69, 70)
(70, 72)
(72, 73)
(73, 74)
(74, 75)
(75, 76)
(76, 77)
(77, 78)
(78, 80)
(80, 81)
(81, 82)
(82, 83)
(83, 84)
(84, 85)
(85, 86)
(86, 88)
(88, 89)
(89, 90)
(90, 91)
(91, 92)
(92, 93)
(93, 94)
(94, 96)
(96, 97)
(97, 98)
(98, 99)
(99, 100)
(100, 101)
(101, 102)
(102, 104)
(104, 105)
(105, 106)
(106, 107)
(107, 108)
(108, 109)
(109, 110)
(110, 112)
(112, 113)
(113, 114)
(114, 115)
(115, 116)
(116, 117)
(117, 118)
(118, 120)
(120, 121)
(121, 122)
(122, 123)
(123

In [9]:
N//P

1